In [13]:
using Plots
using Random
using StatsBase

In [181]:
function transition(f, from, to, frames)
    length(from) == length(to) || error("from and to length must be same")
    for fr in 1:frames
        f(from .+ (to .- from) .*  ((fr - 1) / (frames - 1)))
    end
end

transition (generic function with 1 method)

In [216]:

function animate(;seed=2127365000, n_randomizations=25)
    anim = Plots.Animation()
    rng = MersenneTwister(seed)
    
    lay() = @layout [a{0.6w} b{0.4w}]

    differences = Vector{Float64}()
    
    y = [rand(rng, 5) * 1.5; rand(rng, 5)]
    colors = ["#4B9CD3", "#13294B"]
    x = collect(1:10)
        
    pt_est = mean(y[6:10]) - mean(y[1:5])
    bp = bar(x[1:5], y[1:5], bar_width=0.8, legend=false, color=colors[1], xlab="", xticks=false)
    bar!(x[6:10], y[6:10], bar_width=0.8, color=colors[2])
    
    hp = plot([], [], xlim=(-2 * abs(pt_est), 2 * abs(pt_est)), legend=false, xlab="Sampling dist.",
        ylim=(0, 0.25 * n_randomizations), yticks=false)
    
    frame(anim, plot(bp, hp, layout=lay()))
    
    mean1 = mean(y[1:5])
    mean2 = mean(y[6:10])
    plot!(bp, [0.6, 5.4], fill(mean1, 2), color=colors[1])
    plot!(bp, [5.6, 10.4], fill(mean2, 2), color=colors[2])
    
    frame(anim, plot(bp, hp, layout=lay()))
    
    # fancy animation showing difference
    transition([5.5, min(mean1, mean2)], [10.4, 0], 10) do xy
        bp = bar(x[1:5], y[1:5], bar_width=0.8, legend=false, color=colors[1], xlab="", xticks=false)
        bar!(bp, x[6:10], y[6:10], bar_width=0.8, color=colors[2])
        plot!(bp, [0.6, 5.5], fill(mean1, 2), color=colors[1])
        plot!(bp, [5.5, 10.4], fill(mean2, 2), color=colors[2])
        plot!(bp, [xy[1], xy[1]], [xy[2], xy[2] + abs(mean1 - mean2)], color=:black)
        frame(anim, plot(bp, hp, layout=lay()))
    end
    
    # get rid of animation
    bp = bar(x[1:5], y[1:5], bar_width=0.8, legend=false, color=colors[1], xlab="", xticks=false)
    bar!(bp, x[6:10], y[6:10], bar_width=0.8, color=colors[2])
    plot!(bp, [0.6, 5.5], fill(mean1, 2), color=colors[1])
    plot!(bp, [5.5, 10.4], fill(mean2, 2), color=colors[2])
    
    vline!(hp, [pt_est], color=:black)
    vspan!(hp, [-100, -abs(pt_est)], fillcolor=:gray, linecolor=:gray, alpha=0.25)
    vspan!(hp, [abs(pt_est), 100], fillcolor=:gray, linecolor=:gray, alpha=0.25)
    frame(anim, plot(bp, hp, layout=lay()))

    for i in 1:n_randomizations
        nextx = Random.shuffle(x)
        transition(x, nextx, 10) do xt
            bp = bar(xt[1:5], y[1:5], bar_width=0.8, legend=false, color=colors[1], xlab="", xticks=false)
            bar!(xt[6:10], y[6:10], bar_width=0.8, color=colors[2])
            frame(anim, plot(bp, hp, layout=lay()))
        end
        
        # now, find group means
        mean1 = mean(y[x .≤ 5])
        mean2 = mean(y[x .> 5])
        plot!(bp, [0.6, 5.5], fill(mean1, 2), color=colors[1])
        plot!(bp, [5.5, 10.4], fill(mean2, 2), color=colors[2])
        frame(anim, plot(bp, hp, layout=lay()))
        
        # fancy animation showing difference
        transition([5.5, min(mean1, mean2)], [10.4, 0], 5) do xy
            bp = bar(nextx[1:5], y[1:5], bar_width=0.8, legend=false, color=colors[1], xlab="", xticks=false)
            bar!(nextx[6:10], y[6:10], bar_width=0.8, color=colors[2])

            plot!(bp, [0.6, 5.5], fill(mean1, 2), color=colors[1])
            plot!(bp, [5.5, 10.4], fill(mean2, 2), color=colors[2])
            plot!(bp, [xy[1], xy[1]], [xy[2], xy[2] + abs(mean1 - mean2)], color=:black)
            frame(anim, plot(bp, hp, layout=lay()))
        end
        
        push!(differences, mean(y[nextx .> 5]) - mean(y[nextx .≤ 5]))
        
        pval = mean(abs.(differences) .> abs(pt_est))
        
        # get rid of animated difference
        bp = bar(nextx[1:5], y[1:5], bar_width=0.8, legend=false, color=colors[1], xlab="", xticks=false)
        bar!(nextx[6:10], y[6:10], bar_width=0.8, color=colors[2])
        plot!(bp, [0.6, 5.5], fill(mean1, 2), color=colors[1])
        plot!(bp, [5.5, 10.4], fill(mean2, 2), color=colors[2])
        
        binwidth = abs(pt_est)/10        
        hp = histogram(differences, xlim=(-2 * abs(pt_est), 2 * abs(pt_est)), legend=false,
            bins=(-2 * abs(pt_est)):binwidth:(2 * abs(pt_est)), ylim=(0, 0.25 * n_randomizations), yticks=false,
            xlab="Sampling dist.")
        annotate!((-abs(pt_est) + 0.1, 0.245 * n_randomizations,
                text("p=$(round(pval, digits=2))", family="Open Sans", pointsize=10, halign=:left, valign=:top)))
        vspan!(hp, [-100, -abs(pt_est)], fillcolor=:gray, linecolor=:gray, alpha=0.25)
        vspan!(hp, [abs(pt_est), 100], fillcolor=:gray, linecolor=:gray, alpha=0.25)
        vline!(hp, [pt_est], color=:black)
        frame(anim, plot(bp, hp, layout=lay()))
                     
        x = nextx
    end
    
    anim
end

animate (generic function with 3 methods)

In [ ]:
gif(animate(n_randomizations=200), "permutation.gif")